In [105]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
import re, nltk, random, os, json
import http.client, urllib.request, urllib.parse, urllib.error, base64
from nltk.corpus import stopwords
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG
from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from rank_bm25 import BM25Okapi
import time, os, pickle
# keras
from keras.layers import Input, LSTM, GlobalMaxPooling1D, GlobalAveragePooling1D, Concatenate, Embedding, Multiply, Dot, Dense, Subtract, Activation, SimpleRNN, Flatten, Lambda
from keras.models import Model
from keras import backend as K
import numpy as np


In [2]:
def getVectors(queries):
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '924c1505854b4da4a6144a1cce92937f',
    }
    
    queries = [str(i).replace("\'", "") for i in queries]

    params = urllib.parse.urlencode({})
    
    while True:
        try:
            conn = http.client.HTTPSConnection('api.msturing.org')
    #         conn.request("POST", "/gen/encode?%s" % params, '{"queries": ["how to make gingerbread people (in grams)", "test AI"]}', headers)
            conn.request("POST", "/gen/encode?%s" % params, str({"queries": queries}).replace("\'", "\""), headers)
            response = conn.getresponse()
            data = response.read()
            data = json.loads(data)
            conn.close()
            break
        except Exception as e:
    #         print(data)
            time.sleep(5)
#         print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    return {data[i]['query']:data[i]['vector'] for i in range(len(data))}

In [119]:
stop_words = set(stopwords.words('english'))

regex = re.compile('[^a-zA-Z0-9.]')
#First parameter is the replacement, second parameter is your input string
def preprocessingText(doc):
    doc = regex.sub(' ', doc)
#     doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc.lower()

def evaluate(qrels, ranked_list):
    res = []
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)
    result = {}
    
    
    
    for i in [5, 10 ,15]:
        metric = MSnDCG(xrelnum, grades, cutoff=i)
        result["ndcg@%d" % i] = metric.compute(labeled_ranked_list)
        
        _ranked_list = ranked_list[:i]
        result["p@%d" % i] = len(set.intersection(set(qrels.keys()), set(_ranked_list))) / len(_ranked_list)
        result["r@%d" % i] = len(set.intersection(set(qrels.keys()), set(_ranked_list))) / len(qrels)
        
    return result

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return " ".join([m.group(0) for m in matches]).lower()

def getTermMSvec(all_properties):
    tmp = {}
    for i in range(0, len(all_properties), 20):
        data = getVectors(all_properties[i:i+20])
        for i in data:
            tmp[i] = data[i]
    return tmp

def saveDict(d, name):
    f = open(name,"wb")
    pickle.dump(d,f)
    f.close()
def loadDict(name):
    return pickle.load( open(name, "rb" ) )

In [4]:
df = pd.read_csv("data/AKG/Test Collection/AKGG/akg_standard_akgg_property_rele.csv")
df_action = pd.read_csv("data/AKG/Test Collection/AM/akg_standard_am_verb_object_rele.csv")

with open("data/AKG/Formal Run Topics/AKGG_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, query, entity, entityType, action = [], [], [], [], []
    for p in data['queries']:
        qid.append(p['queryId'])
        query.append(p['query'])   
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        action.append(p['action'])
topic = pd.DataFrame({"query_id": qid, "query": query, "entity": entity, "entityType": entityType, "action":action})
for c in ["query", "entityType", "action", "entity"]:
    topic[c] = topic[c].str.lower().replace("\'", "")
    
df = df.merge(topic, how="inner", on="query_id")
# df['query'] = df[['action', 'entity', 'entityType']].astype(str).apply(' '.join, axis=1)

In [5]:
trainIds, testIds = [], []
for name, group in df.groupby("entityType"):
    if group.query_id.nunique() > 1:
        ids = list(group.query_id.unique())
        mid = int(group.query_id.nunique() / 2)
        trainIds.extend(ids[:mid])
        testIds.extend(ids[mid:])
    else:
        ids = list(group.query_id.unique())
        trainIds.extend(ids)

In [6]:
import json

with open("data/AKG/Formal Run Topics/AM_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, entityurl, entity, entityType = [], [], [], []
    for p in data['queries'][0]:
        qid.append(p['queryId'])
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        entityurl.append(p['entityurl'])
am_topic = pd.DataFrame({"query_id": qid, "url": entityurl, "entity": entity, "entityType": entityType})

In [14]:
df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['headline'] = df_wiki['headline'].str.replace("\n", "")
df_wiki['title'] = df_wiki['title'].str.replace("How to ", "")

df_wiki['overview'] = [preprocessingText(str(i)) for i in df_wiki['overview']]
df_wiki['headline'] = [preprocessingText(str(i)) for i in df_wiki['headline']]
df_wiki['text'] = [preprocessingText(str(i)) for i in df_wiki['text']]
df_wiki['sectionLabel'] = [preprocessingText(str(i)) for i in df_wiki['sectionLabel']]
df_wiki['title'] = [preprocessingText(str(i)) for i in df_wiki['title']]
df_wiki['title'] = [i if not i[-1].isdigit() else i[:-1] for i in df_wiki['title']]

In [17]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value
with open("data/AKG/Participants Runs/AKGG/akgg-formalrun-cuis.json") as json_file:
    data = json.load(json_file)
    run = AutoVivification()
    for p in data['runs']:
        for res in p['results']:
            for prop in res['properties']:
                run[p['runid']][str(res['queryid'])][str(prop['property'])] = prop['rank']

qids = []
props = []
for qid in run['1']:
    tmp = list(run['1'][str(qid)].keys())
    qids.extend([int(qid)] * len(tmp))
    props.extend(tmp)
df_run = pd.DataFrame({"query_id": qids, "property": props})
df_run = df_run.merge(topic, how="left", on="query_id")

In [18]:
import collections
qrel = collections.defaultdict(dict)
for qid, prop, label in df[['query_id', 'property', 'rele_label']].values:
    qrel[str(qid)][str(prop)] = int(label)

In [20]:
dfp = df[["query_id", "entityType", "property"]].append(df_run[["query_id", "entityType", "property"]])
dfp = dfp[dfp.query_id.isin(trainIds)]
type2prop = dfp.groupby("entityType")['property'].unique().to_dict()
prop2popularity = dfp.groupby("property").size().to_dict()
type2prop2popularity = dfp.groupby(["entityType", "property"]).size().to_dict()

In [ ]:
terms = []
for c in [ 'entity', 'entityType', 'action']:
    for i in df[c].unique().tolist():
        terms.append(i.replace("\'", ""))
terms.extend([camel_case_split(i) for i in df.property.unique()])
term2MSvec = getTermMSvec(terms)

In [25]:
def get_pretrain_embeddings(path, word_index, EMBEDDING_DIM=100):
    MAX_NUM_WORDS = len(word_index)
    BASE_DIR = path + 'data/'
    GLOVE_DIR = os.path.join(BASE_DIR, 'w2v')
    print('Indexing word vectors.')

    embeddings_index = {}
    with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf-8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, 'f', sep=' ')
            embeddings_index[word] = coefs

    print('Found %s word vectors.' % len(embeddings_index))
    print('Preparing embedding matrix.')

    # prepare embedding matrix
    num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    found = 0
    for word, i in word_index.items():
        if i > MAX_NUM_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if embedding_vector.shape[0] == 0:
                continue
            embedding_matrix[i] = embedding_vector
            found += 1

    print("Token num: %d, Found Tokens: %d" % (len(word_index), found))

    # load pre-trained word embeddings into an Embedding layer
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                embeddings_initializer=Constant(embedding_matrix))

    return embedding_layer
MAX_NUM_WORDS = 1000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(list(term2MSvec.keys()))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
path = "/Users/jarana/workspace/WikiHow-Task-Based/"
max_words = len(word_index)
embedding_layer = get_pretrain_embeddings(path, word_index)

Found 993 unique tokens.
Indexing word vectors.


W1209 14:28:57.823349 4570330560 deprecation_wrapper.py:119] From /Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Found 400000 word vectors.
Preparing embedding matrix.
Token num: 993, Found Tokens: 963


In [28]:
queries = []
for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
    queries.append(row['action'] + " " + row['entity'] + " " + row['entityType'])
properties = [camel_case_split(i) for i in df['property'].unique()]

MAX_NUM_WORDS = 100000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus + queries + properties)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

NameError: name 'corpus' is not defined

In [19]:
path = "/Users/jarana/workspace/WikiHow-Task-Based/"
max_words = len(word_index)
embedding_layer = get_pretrain_embeddings(path, word_index)

NameError: name 'word_index' is not defined

In [ ]:
def getWord2Vec(embedding_layer, MAX_SEQUENCE_LENGTH=10000):
    q_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
    q_emb = GlobalMaxPooling1D()(embedding_layer(q_inp))
    model = Model(q_inp, q_emb)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model
word2vec = getWord2Vec(embedding_layer)
prop2Glovevec = {p: word2vec.predict(pad_sequences(tokenizer.texts_to_sequences([camel_case_split(p)]), maxlen=10000))[0] for p in dfp.property.unique()}
qid2Glovevec = {}
for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
    q = row['action'] + " " + row['entity'] + " " + row['entityType']
    qid2Glovevec[row['query_id']] = word2vec.predict(pad_sequences(tokenizer.texts_to_sequences([q]), maxlen=10000))[0] 


In [21]:
dfw = df_wiki[df_wiki.title.str.contains("|".join(df.entity.unique().tolist()))]

In [23]:
entityInWiki = []
for e in df.entity.unique():
    tmp = dfw[dfw.title.str.contains(e)]
    if len(tmp) > 0:
        entityInWiki.append(e)

In [19]:
# corpus = []
# for name, group in df_wiki2.groupby("title"):
#     title = name
#     overview = group['overview'].unique().tolist()
#     text = group['text'].unique().tolist()
#     headline = group['headline'].unique().tolist()
#     sectionLabel = group['sectionLabel'].unique().tolist()
# #     doc = " \t ".join([title] + overview + sectionLabel + headline + text)
#     doc = ". ".join([title] + [str(i) for i in headline])
#     corpus.append(doc)
# corpus = [preprocessingText(i) for i in corpus]
corpus = df_wiki.title.unique().tolist()
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# qid2doc2MSvec = {}
# for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
#     query = row['action'] + " " + row['entity'] + " " + row['entityType']
#     query = query.replace("thing", "")
#     tokenized_query = query.split(" ")
#     doc = bm25.get_top_n(tokenized_query, corpus, n=1)
#     data = getVectors(doc)
#     qid2doc2MSvec[row['query_id']] = data[doc[0]]

In [460]:
for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
    query = row['action'] + " " + row['entity'] + " " + row['entityType']
    query = query.replace("thing", "")
    tokenized_query = query.split(" ")
    doc = bm25.get_top_n(tokenized_query, corpus, n=5)
#     data = getVectors(doc)

In [25]:
# load : qid2MSvec.pkl
import pickle

qid2MSvec = pickle.load( open( "qid2MSvec.pkl", "rb" ) )
# qid2MSvec = {}
# for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
#     query = row['action'] + " " + row['entity']
#     if row['query_id'] in qid2MSvec:
#         continue
#     if row['entity'] not in entityInWiki:
#         data = getVectors([query])
#         qid2MSvec[row['query_id']] = list(data.values())
#         continue        
# #     titles = [str(i).replace("\'", "") for i in dfw[dfw.title.str.contains(row['entity'])].text.unique().tolist()]
#     titles = dfw[dfw.title.str.contains(row['entity'])].title.unique().tolist()
#     data = getTermMSvec([query] + titles)
#     rank = {}
#     for i in data:
#         if i == query:
#             continue
#         rank[i] = cosine_similarity([data[query]],jj [data[i]])[0][0]
#     best_title = sorted(rank.items(), key=lambda x: x[1])[-1][0]
#     sentences = []
#     tokens = " ".join(dfw[dfw.title == best_title].text.tolist()).split()
#     for i in range(0, len(tokens), 10):
#         sentences.append(" ".join(tokens[i:i+10]))
#     data = getTermMSvec([query] + sentences)
#     qid2MSvec[row['query_id']] = list(data.values())


In [78]:
# Get questions
import glob
from run_scraper import crawl
from tqdm.autonotebook import tqdm

# finish = []
# for file in glob.glob("akg/*"):
#     finish.append(file.split("/")[1].replace("_", " ").replace(".csv", ""))

bestTitle2qid = {}
for idx, row in tqdm(df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows(), total=229):
    query = row['action'] + " " + row['entity']
    if row['entity'] not in entityInWiki:
        continue        
    titles = dfw[dfw.title.str.contains(row['entity'])].title.unique().tolist()
    data = getTermMSvec([query] + titles)
    rank = {}
    for i in data:
        if i == query:
            continue
        rank[i] = cosine_similarity([data[query]], [data[i]])[0][0]
    best_title = sorted(rank.items(), key=lambda x: x[1])[-1][0]
    bestTitle2qid[best_title] = row['query_id']
#     try:
#         crawl(best_title, "akg/", "no_question")
#     except:
#         print(best_title)
            


In [120]:
import glob
# qid2questions2MSvec = {}
# for file in glob.glob("akg/*"):
#     title = file.split("/")[1].replace("_", " ").replace(".csv", "")
#     lines = open(file).read().splitlines()
#     questions = []
#     for i in lines[1:]:
#         questions.extend(i.split(";"))
#     title2questions[title] = list(set(questions))
#     data = getVectors([title] + list(set(questions)) )
#     qid2questions2MSvec[bestTitle2qid[title]] = list(data.values())
qid2questions2MSvec = loadDict("qid2questions2MSvec")

In [28]:
# qid2query2MSvec = {}
# queries = []
# for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
#     query = preprocessingText(row['action'] + " " + row['entity'] + " " + row['entityType'])
#     qid2query2MSvec[row['query_id']] = getVectors([query])[query]
qid2query2MSvec = pickle.load( open( "qid2query2MSvec.pkl", "rb" ) )

In [29]:
# properties = []
# for i in run["1"]:
#     properties.extend(run["1"][i].keys())
# properties = list(set(properties + df.property.unique().tolist()))
# prop2MSvec = getTermMSvec(properties)
prop2MSvec = pickle.load( open( "prop2MSvec.pkl", "rb" ) )

In [ ]:
# CLF with Question MS vectors
MAX_SEQUENCE_LENGHT = int(np.max([len(qid2MSvec[i]) for i in qid2MSvec])) + 1
# def runDocEval(qid2MSvec, prop2MSvec):
class CLF():
    def __init__(self):

        self.queryInput = Input(shape=(MAX_SEQUENCE_LENGHT, 100,))
        self.propPosInput = Input(shape=(100,))
        lstm = LSTM(100)

        queryEmbeddingLayer = Dense(100, name="uEmb")
        propEmbeddingLayer = Dense(100, name="iEmb")

#         qEmb = queryEmbeddingLayer(lstm(self.queryInput))
        qEmb = queryEmbeddingLayer(self.queryInput)
        pEmb = propEmbeddingLayer(self.propPosInput)

        dense = Dense(1, activation="sigmoid")

        pred = Multiply()([qEmb, pEmb])
        
        self.pred = dense(GlobalMaxPooling1D()(pred))
#         self.pred = dense(pred)

        self.model = Model(inputs=[self.queryInput, self.propPosInput], outputs=self.pred)
        self.model.compile(optimizer="adam", loss="binary_crossentropy")

    def generate_train_data(self, df):
        x_query, x_pos_prop, y = [], [], []
        for name, group in df.groupby("query_id"):
            cand_pos_prop = group.property.tolist()
            rele2prop = group[['rele_label', 'property']].groupby("rele_label")["property"].apply(list).to_dict()
            for idx, row in group.iterrows():
                if name in qid2questions2MSvec:
                    x_query.append(qid2questions2MSvec[name] + [qid2query2MSvec[name]])
                else:
                    x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
#                 x_query.append([qid2query2MSvec[name]])
                x_pos_prop.append(prop2MSvec[row['property']])
                y.append(1)
        
            cand_neg_prop = list(set.difference(set(type2prop[row['entityType']]), set(cand_pos_prop)))
#             cand_neg_prop = list(set.difference(set(df.property.unique()), set(cand_pos_prop)))
            for neg_prop in cand_neg_prop:
                if name in qid2questions2MSvec:
                    x_query.append(qid2questions2MSvec[name] + [qid2query2MSvec[name]])
                else:
                    x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
#                 x_query.append([qid2query2MSvec[name]])
                x_pos_prop.append(prop2MSvec[neg_prop])
                y.append(0)

        x_query = pad_sequences(x_query,dtype='float32', maxlen=MAX_SEQUENCE_LENGHT)
        x_pos_prop = np.array(x_pos_prop)
        return [x_query, x_pos_prop], np.array(y)

df_train = df[df.query_id.isin(trainIds)]
bpr = CLF()
x_train, y_train = bpr.generate_train_data(df_train)
for i in range(10):
    test_num = 0
    history = bpr.model.fit(x_train, y_train, verbose=0)
    res = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg"] for j in [5, 10 ,15]}
    for idx, row in df[['query_id', 'entity', 'action', 'entityType', 'query']].drop_duplicates().iterrows():
        if row['query_id'] not in testIds:
            continue
        if row['entity'] not in entityInWiki:
                continue
        test_num += 1
        qrels = qrel[str(row['query_id'])]
        cand_properties = type2prop[row['entityType']]
#         cand_properties = list(prop2MSvec.keys())
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))

        rank = {}
        for p in cand_properties:
            if row['query_id'] in qid2questions2MSvec:
                score = bpr.model.predict([pad_sequences([qid2questions2MSvec[row['query_id']] + [qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
            else:    
                score = bpr.model.predict([pad_sequences([qid2MSvec[row['query_id']] + [qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
#             score = bpr.model.predict([pad_sequences([[qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
            rank[p] = score
#         rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
        our = evaluate(qrels, rank)
        for key in res:
            res[key].append(our[key])
    for key in ['ndcg@15']:
        print(key, np.mean(res[key]))
# runDocEval(qid2MSvec, prop2MSvec)

In [39]:
# Most recent and best performing model
MAX_SEQUENCE_LENGHT = int(np.max([len(qid2MSvec[i]) for i in qid2MSvec])) + 1
# def runDocEval(qid2MSvec, prop2MSvec):
class CLF():
    def __init__(self):

        self.queryInput = Input(shape=(MAX_SEQUENCE_LENGHT, 100,))
        self.propPosInput = Input(shape=(100,))
        lstm = LSTM(100)

        queryEmbeddingLayer = Dense(100, name="uEmb")
        propEmbeddingLayer = Dense(100, name="iEmb")

#         qEmb = queryEmbeddingLayer(lstm(self.queryInput))
        qEmb = queryEmbeddingLayer(self.queryInput)
        pEmb = propEmbeddingLayer(self.propPosInput)

        dense = Dense(1, activation="sigmoid")

        pred = Multiply()([qEmb, pEmb])
        
        self.pred = dense(GlobalMaxPooling1D()(pred))
#         self.pred = dense(pred)

        self.model = Model(inputs=[self.queryInput, self.propPosInput], outputs=self.pred)
        self.model.compile(optimizer="adam", loss="binary_crossentropy")

    def generate_train_data(self, df):
        x_query, x_pos_prop, y = [], [], []
        for name, group in df.groupby("query_id"):
            cand_pos_prop = group.property.tolist()
            rele2prop = group[['rele_label', 'property']].groupby("rele_label")["property"].apply(list).to_dict()
            for idx, row in group.iterrows():
                x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
#                 x_query.append([qid2query2MSvec[name]])
                x_pos_prop.append(prop2MSvec[row['property']])
                y.append(1)

            cand_neg_prop = list(set.difference(set(type2prop[row['entityType']]), set(cand_pos_prop)))
#             cand_neg_prop = list(set.difference(set(df.property.unique()), set(cand_pos_prop)))
            for neg_prop in cand_neg_prop:
                x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
#                 x_query.append([qid2query2MSvec[name]])
                x_pos_prop.append(prop2MSvec[neg_prop])
                y.append(0)

        x_query = pad_sequences(x_query,dtype='float32', maxlen=MAX_SEQUENCE_LENGHT)
        x_pos_prop = np.array(x_pos_prop)
        return [x_query, x_pos_prop], np.array(y)

df_train = df[df.query_id.isin(trainIds)]
bpr = CLF()
x_train, y_train = bpr.generate_train_data(df_train)
for i in range(10):
    test_num = 0
    history = bpr.model.fit(x_train, y_train, verbose=0)
    res = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg"] for j in [5, 10 ,15]}
    for idx, row in df[['query_id', 'entity', 'action', 'entityType', 'query']].drop_duplicates().iterrows():
        if row['query_id'] not in testIds:
            continue
        if row['entity'] not in entityInWiki:
                continue
        test_num += 1
        qrels = qrel[str(row['query_id'])]
        cand_properties = type2prop[row['entityType']]
#         cand_properties = list(prop2MSvec.keys())
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))

        rank = {}
        for p in cand_properties:
            score = bpr.model.predict([pad_sequences([qid2MSvec[row['query_id']] + [qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
#             score = bpr.model.predict([pad_sequences([[qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
            rank[p] = score
#         rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
        our = evaluate(qrels, rank)
        for key in res:
            res[key].append(our[key])
    for key in ['ndcg@15']:
        print(key, np.mean(res[key]))
# runDocEval(qid2MSvec, prop2MSvec)

Epoch 1/1
3785/3785 [==============================] - 5s 1ms/step - loss: 0.6863
ndcg@15 0.5716840359523024
Epoch 1/1
3785/3785 [==============================] - 4s 951us/step - loss: 0.6256
ndcg@15 0.5819940098938364
Epoch 1/1
3785/3785 [==============================] - 4s 1ms/step - loss: 0.5937
ndcg@15 0.5608354134879565
Epoch 1/1
3785/3785 [==============================] - 4s 1ms/step - loss: 0.5690
ndcg@15 0.5877968688552294
Epoch 1/1
3785/3785 [==============================] - 4s 1ms/step - loss: 0.5461
ndcg@15 0.5932514972455951
Epoch 1/1
3785/3785 [==============================] - 4s 1ms/step - loss: 0.5229
ndcg@15 0.5888175164402596
Epoch 1/1
3785/3785 [==============================] - 4s 1ms/step - loss: 0.5063
ndcg@15 0.5862739048630199
Epoch 1/1
3785/3785 [==============================] - 4s 1ms/step - loss: 0.4848
ndcg@15 0.5835915711406858
Epoch 1/1
3785/3785 [==============================] - 4s 1ms/step - loss: 0.4604
ndcg@15 0.6007156661227928
Epoch 1/1
3785/37

In [42]:
def runEval(qid2vec, prop2vec):
    class CLF():
        def __init__(self):

            self.queryInput = Input(shape=(100,))
            self.propPosInput = Input(shape=(100,))

            queryEmbeddingLayer = Dense(100, name="uEmb")
            propEmbeddingLayer = Dense(100, name="iEmb")

            qEmb = queryEmbeddingLayer(self.queryInput)
            pEmb = propEmbeddingLayer(self.propPosInput)

            dense = Dense(1, activation="sigmoid")

            pred = Multiply()([qEmb, pEmb])
            self.pred = dense(pred)

            self.model = Model(inputs=[self.queryInput, self.propPosInput], outputs=self.pred)
            self.model.compile(optimizer="adam", loss="binary_crossentropy")
    #         self.predictor = Model([self.queryInput, self.propPosInput], [pDot])
    #         self.predictor = Model([self.entityInput, self.entityTypeInput, self.actionInput, self.propPosInput], [pDot])
        def generate_train_data(self, df):
            x_query, x_entity, x_type, x_action, x_pos_prop, x_neg_prop, y = [], [], [], [], [], [], []
            for name, group in df.groupby("query_id"):
                cand_pos_prop = group.property.tolist()
                rele2prop = group[['rele_label', 'property']].groupby("rele_label")["property"].apply(list).to_dict()

                for idx, row in group.iterrows():

                    x_query.append(qid2vec[name])
                    x_pos_prop.append(prop2vec[row['property']])
                    y.append(1)

                cand_neg_prop = list(set.difference(set(type2prop[row['entityType']]), set(cand_pos_prop)))
    #             cand_neg_prop = list(set.difference(set(df.property.unique()), set(cand_pos_prop)))
                for neg_prop in cand_neg_prop:
                    x_query.append(qid2vec[name])
                    x_pos_prop.append(prop2vec[neg_prop])
                    y.append(0)

            x_query = np.array(x_query)
            x_entity = np.array(x_entity)
            x_type = np.array(x_type)
            x_action = np.array(x_action)
            x_pos_prop = np.array(x_pos_prop)
            x_neg_prop = np.array(x_neg_prop)
    #         return [x_entity, x_type, x_action, x_pos_prop, x_neg_prop], np.ones(len(x_query))
            return [x_query, x_pos_prop], np.array(y)

    df_train = df[df.query_id.isin(trainIds)]
    bpr = CLF()

    for i in range(10):
        x_train, y_train = bpr.generate_train_data(df_train)
        print(x_train[0].shape)
        res = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg"] for j in [5, 10 ,15]}
        history = bpr.model.fit(x_train, y_train, verbose=1)
        for idx, row in df[['query_id', 'entity', 'action', 'entityType', 'query']].drop_duplicates().iterrows():
            if row['query_id'] not in testIds:
                continue
            if row['entity'] not in entityInWiki:
                continue
            qrels = qrel[str(row['query_id'])]
            cand_properties = type2prop[row['entityType']]
#             cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))

            rank = {}
            for p in cand_properties:
                score = bpr.model.predict([[qid2vec[row['query_id']]], [prop2vec[p]]])[0][0]
                rank[p] = score
            rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
            our = evaluate(qrels, rank)
#             res.append(our)
            
            for key in res:
                res[key].append(our[key])
        for key in ['ndcg@15']:
            print(key, np.mean(res[key]))


In [43]:
runEval(qid2query2MSvec, prop2MSvec)

(3785, 100)
Epoch 1/1
3785/3785 [==============================] - 1s 384us/step - loss: 0.6791
ndcg@15 0.557532011532777
(3785, 100)
Epoch 1/1
3785/3785 [==============================] - 0s 80us/step - loss: 0.6105
ndcg@15 0.5729153478538512
(3785, 100)
Epoch 1/1
3785/3785 [==============================] - 0s 77us/step - loss: 0.5699
ndcg@15 0.5713174830915543
(3785, 100)
Epoch 1/1
3785/3785 [==============================] - 0s 77us/step - loss: 0.5315
ndcg@15 0.5656313224599708
(3785, 100)
Epoch 1/1
3785/3785 [==============================] - 0s 80us/step - loss: 0.4994
ndcg@15 0.5702425934497766
(3785, 100)
Epoch 1/1
3785/3785 [==============================] - 0s 80us/step - loss: 0.4669
ndcg@15 0.5789620958972643
(3785, 100)
Epoch 1/1
3785/3785 [==============================] - 0s 80us/step - loss: 0.4361
ndcg@15 0.5745994579896394
(3785, 100)
Epoch 1/1
3785/3785 [==============================] - 0s 80us/step - loss: 0.4061
ndcg@15 0.5675766103516355
(3785, 100)
Epoch 1/1
37

In [188]:
def bpr_triplet_loss(X):
    positive_item_latent, negative_item_latent = X

    loss = 1 - K.log(K.sigmoid(positive_item_latent - negative_item_latent))

    return loss


def identity_loss(y_true, y_pred):
    return K.mean(y_pred - 0 * y_true)


class BPR():
    def __init__(self):

        self.queryInput = Input(shape=(100,))
#         self.entityTypeInput = Input(shape=(100,))
#         self.actionInput = Input(shape=(100,))
        
        self.propPosInput = Input(shape=(100,))
        self.propNegInput = Input(shape=(100,))

        queryEmbeddingLayer = Dense(100, name="uEmb")
        propEmbeddingLayer = Dense(100, name="iEmb")

        self.qEmb = queryEmbeddingLayer(self.queryInput)
        self.pEmb = propEmbeddingLayer(self.propPosInput)
        self.nEmb = propEmbeddingLayer(self.propNegInput)

        pDot = Dot(axes=-1)([self.qEmb, self.pEmb])
        nDot = Dot(axes=-1)([self.qEmb, self.nEmb])

#         peDot = Dot(axes=-1)([self.dentityInput, self.propPosInput])
#         ptDot = Dot(axes=-1)([self.entityTypeInput, self.propPosInput])
#         paDot = Dot(axes=-1)([self.actionInput, self.propPosInput])

#         neDot = Dot(axes=-1)([self.entityInput, self.propNegInput])
#         ntDot = Dot(axes=-1)([self.entityTypeInput, self.propNegInput])
#         naDot = Dot(axes=-1)([self.actionInput, self.propNegInput])
        
#         pDot = Concatenate()([self.entityInput, self.entityTypeInput, self.actionInput, self.propPosInput])
#         nDot = Concatenate()([self.entityInput, self.entityTypeInput, self.actionInput, self.propNegInput])
        
        dense = Dense(1, activation="sigmoid")
        
#         pDot = Multiply()([self.queryInput, self.propPosInput])
#         nDot = Multiply()([self.queryInput, self.propNegInput])
        
#         pDot = Dot(axes=-1)([self.queryInput, self.propPosInput])
#         nDot = Dot(axes=-1)([self.queryInput, self.propNegInput])
        
        pDot = dense(pDot)
        nDot = dense(nDot)
#         pred = Multiply()([q_emb, t_emb])
        #
        # diff = Subtract()([pDot, nDot])
        #
        lammbda_output = Lambda(bpr_triplet_loss, output_shape=(1,))
        self.pred = lammbda_output([pDot, nDot])

        self.model = Model(inputs=[self.queryInput, self.propPosInput, self.propNegInput], outputs=self.pred)
#         self.model = Model(inputs=[self.entityInput, self.entityTypeInput, self.actionInput, self.propPosInput, self.propNegInput], outputs=self.pred)
#         self.model = Model(inputs=[self.entityInput, self.entityTypeInput, self.actionInput, self.propPosInput, self.propNegInput], outputs=self.pred)

        self.model.compile(optimizer="adam", loss=identity_loss)
        self.predictor = Model([self.queryInput, self.propPosInput], [pDot])
#         self.predictor = Model([self.entityInput, self.entityTypeInput, self.actionInput, self.propPosInput], [pDot])
    def generate_train_data(self, df):
        x_query, x_entity, x_type, x_action, x_pos_prop, x_neg_prop, y = [], [], [], [], [], [], []
        for name, group in df.groupby("query_id"):
            cand_pos_prop = group.property.tolist()
            rele2prop = group[['rele_label', 'property']].groupby("rele_label")["property"].apply(list).to_dict()

            
            for idx, row in group.iterrows():
                
                propLabel = row['rele_label']
                for rele in rele2prop:
                    if propLabel > rele:
                        for neg_prop in rele2prop[rele]:
                            x_query.append(qid2MSvec[name])
                            x_pos_prop.append(prop2MSvec[row['property']])
                            x_neg_prop.append(prop2MSvec[neg_prop])
                
                cand_neg_prop = list(set.difference(set(type2prop[row['entityType']]), set(cand_pos_prop)))
                for neg_prop in cand_neg_prop:
                    x_query.append(qid2MSvec[name])
                    x_pos_prop.append(prop2MSvec[row['property']])
                    x_neg_prop.append(prop2MSvec[neg_prop])
                
#                 cand_neg_prop = list(set.difference(set(df.property.unique()), set(cand_pos_prop)))
#                 for neg_prop in cand_neg_prop:
#                     x_query.append(qid2MSvec[name])
                    x_pos_prop.append(prop2MSvec[row['property']])
                    x_neg_prop.append(prop2MSvec[neg_prop])
                            
#                 cand_neg_prop = type2prop[row['entityType']]
#                 cand_neg_prop = df.property.unique().tolist()

#                 for n in range(int(row['rele_label'])):
#                 for n in range(1):
#                     if int(row['rele_label']) < 3:
#                         break
#                     x_entity.append(entity2MSvec[row['entity']])
#                     x_type.append(type2MSvec[row['entityType']])
#                     x_action.append(action2MSvec[row['action'].replace("\'", "")])
#                     x_query.append(qid2MSvec[name])
#                     x_pos_prop.append(prop2MSvec[row['property']])
#                     neg_prop = random.choice(cand_neg_prop)
#                     while neg_prop in cand_pos_prop:
#                         neg_prop = random.choice(cand_neg_prop)
#                     x_neg_prop.append(prop2MSvec[neg_prop])
        x_query = np.array(x_query)
        x_entity = np.array(x_entity)
        x_type = np.array(x_type)
        x_action = np.array(x_action)
        x_pos_prop = np.array(x_pos_prop)
        x_neg_prop = np.array(x_neg_prop)
#         return [x_entity, x_type, x_action, x_pos_prop, x_neg_prop], np.ones(len(x_query))
        return [x_query, x_pos_prop, x_neg_prop], np.ones(len(x_query))
        
        
# print(x_query)
df_train = df[df.query_id.isin(trainIds)]
bpr = BPR()


for i in range(10):
    x_train, y_train = bpr.generate_train_data(df_train)
    print(x_train[0].shape)
    history = bpr.model.fit(x_train, y_train, verbose=1)
    res = []
    for idx, row in df[['query_id', 'entity', 'action', 'entityType', 'query']].drop_duplicates().iterrows():
        if row['query_id'] not in testIds:
            continue
        qrels = qrel[str(row['query_id'])]
#         cand_properties = type2prop[row['entityType']]
        cand_properties = type2prop[row['entityType']]
        

        rank = {}
        for p in cand_properties:
#             score = bpr.predictor.predict([[type2MSvec[row['entityType']]], [prop2MSvec[p]]])[0][0]
            score = bpr.predictor.predict([[qid2MSvec[row['query_id']]], [prop2MSvec[p]]])[0][0]
#             score = bpr.predictor.predict([[x_query], [x_property]])[0][0]
            rank[p] = score
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]
        our = evaluate(qrels, rank)

        res.append(our)
    print(history.history["loss"][0], np.mean(res))


(40586,)


ValueError: Error when checking input: expected input_13 to have shape (100,) but got array with shape (1,)

In [186]:
res = []
res2 = []
for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
    if row['query_id'] not in testIds:
        continue
    if row['entity'] not in entityInWiki:
        continue
    qrels = qrel[str(row['query_id'])]
    cand_properties = type2prop[row['entityType']]
    
    rank = {}
    for p in cand_properties:
#         score = bpr.predictor.predict([[type2MSvec[row['entityType']]], [prop2MSvec[p]]])[0][0]
#         rank[p] = score
#         score = cosine_similarity([type2MSvec[row['entityType']]], [prop2MSvec[p]])[0][0]
#         rank[p] = score
#         if (row['entityType'], p) not in type2prop2popularity:
#             rank[p] = -99999
#         else:
#         rank[p] = type2prop2popularity[(row['entityType'], p)]
#         rank[p] = prop2popularity[p]
        rank[p] = 1
    rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
    our = evaluate(qrels, rank)
#     our = evaluate(qrels, cand_properties)
    base = evaluate(qrels, list(run["1"][str(row["query_id"])].keys()))

    res.append(our)
    res2.append(base)
# print(np.mean(res))
# print(np.mean(res2))
# for i in res2:
#     print(key, np.mean(res2[key]))

keys = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg"] for j in [5, 10 ,15]}
for r in [res, res2]:
    for k in keys:
        print(k, np.mean([i[k] for i in r]))

p@5 0.6377358490566039
p@10 0.6121593291404612
p@15 0.6090146750524109
r@5 0.21082034063385194
r@10 0.40143147962953435
r@15 0.5702095162757292
ndcg@5 0.4251717300658441
ndcg@10 0.487037580220878
ndcg@15 0.5452583811427126
p@5 0.671698113207547
p@10 0.6405660377358491
p@15 0.6594339622641509
r@5 0.21764968417489006
r@10 0.4063436619523424
r@15 0.6031397662478625
ndcg@5 0.47091777226069426
ndcg@10 0.5126074197810446
ndcg@15 0.5772479732667619


In [175]:
np.mean([len(qrel[i]) for i in qrel])

16.467248908296945

In [ ]:
p@5 0.732075471698113
p@10 0.6970649895178196
p@15 0.6392033542976939
r@5 0.23423827318257484
r@10 0.44957616343065254
r@15 0.5939206468091041
ndcg@5 0.5158856370667873
ndcg@10 0.5625110757973671
ndcg@15 0.5977080891368343